In [10]:
from tabulate import tabulate
from datetime import datetime
from openpyxl import load_workbook
from pprint import pprint
import os
from simple_smartsheet import Smartsheet
from simple_smartsheet.models import Sheet, Column, Row, Cell
import pandas as pd


In [11]:
from players import *

In [12]:
def get_indexes_from_xl_file():
	wb = load_workbook(filename = 'Handicap Index Course Handicap Report.xlsx')
	# print(wb)
	ws = wb['Sheet1']
	# print(ws)

	#Retrieve the player's indexes, compute handicaps
	for i in range(2, 21):
		# print(i)
		golfer_name = (ws.cell(row=i, column=3).value)
		# print(golfer_name)
		h_i = (float(ws.cell(row=i, column=4).value))


		#Update handicap info in Player Class, set playing to no.
		for player in player_list:
			if golfer_name == player.ghin_name:
				player.h_i = h_i
				player.class_tpc_white_70()
				player.class_tpc_white_72()
				player.class_cwv_white_71()
				player.playing = False
				# print (f" {golfer_name}, hi: {player.h_i}, tpc 70: {player.handicap_tpc_70}, tpc 72: {player.handicap_tpc_72} cwv: {player.handicap_cwv_71}, status = {player.playing}")


In [13]:
def update_player_status(day_of_play):
	TOKEN = '0b2269ovtwyvi1bmbqm15c2kg9'
	smartsheet = Smartsheet(TOKEN)
	# id=1798700143011716	'Jack Sign-Up'
	sheet = smartsheet.sheets.get(id=1798700143011716)

	#create list of players for specific date
	monday_list = []
	wednesday_list = []
	friday_list = [] 
	
	for row in sheet.rows:
	    player = row.get_cell("Player").value
	    monday_play = row.get_cell("Mon?").value
	    if monday_play: monday_list.append(player)

	    wednesday_play = row.get_cell("Wed?").value
	    if wednesday_play: wednesday_list.append(player)

	    friday_play = row.get_cell("Fri?").value
	    if friday_play:
	    	# print (f"{k}. {player} on Friday")
	    	# k += 1
	    	friday_list.append(player)	

	if day_of_play == 'M' or day_of_play == 'm':
		for player in player_list:
			if player.signup_name in monday_list:
				# print(f"{i}. {player.signup_name}")
				# i += 1
				player.playing = True

	elif day_of_play == 'W' or day_of_play == 'w':
		i = 1
		for player in player_list:
			if player.signup_name in wednesday_list:
				# print(f"{i:02}. {player.signup_name}")
				# i += 1
				player.playing = True

	elif day_of_play == 'F' or day_of_play == 'f':
		i = 1
		for player in player_list:
			if player.signup_name in friday_list:
				# print(f"{i}. {player.signup_name}")
				# i += 1
				player.playing = True
	
	else: print('Bad choice of Day')	


In [14]:
def print_results_new(tpc_min_70,tpc_min_72, cwv_min):  
	today = datetime.datetime.now() #was just datetime.now, but import of datetime in players.py changed that....
	today = today.strftime("%B %d, %Y")

	results_list = []

	choice = input ("[T]PC or [C]WV? > ")

	if choice == 'T' or choice == 't':
		for player in player_list:
			if player.playing:
				results = [player.signup_name, player.h_i, player.handicap_tpc_70, player.handicap_tpc_70 - tpc_min_70] #, player.handicap_tpc_72, player.handicap_tpc_72 - tpc_min_72]
				results_list.append(results)

		results_list.sort()
		# results_list.insert(0,["   ", "   ",  "TPC HCP", "TPC-70", "TPC HCP", "TPC-72"])
		# results_list.insert(1,["Name", "Index","Par 70", "Strks", "Par 72", "Strks"])
		headers = ["\nName", "\nIndex", "TPC HCP\nWhites", "TPC\n Strokes"] #, "TPC HCP\n Par 72", "TPC-72\nStrokes" ]
		column_print_alignment = list(("right","right","right","right"))# , "right", "right"))
		course_info = f"TPC 70: CR = {tpc_rating_white_70} | SR = {tpc_slope_white_70} | Par = {tpc_hcp_70}"
		# course_info = course_info + f"TPC 72: CR = {tpc_rating_white_72} | SR = {tpc_slope_white_72} | Par = {tpc_hcp_72}"

	elif choice == 'C' or choice == 'c':
		for player in player_list:
			if player.playing:
				results = [player.signup_name, player.h_i, player.handicap_cwv_71, player.handicap_cwv_71-cwv_min]
				results_list.append(results)

		results_list.sort()
		# results_list.insert(0,["   ", "   ", "CWV HCP", "CWV"])
		# results_list.insert(1,["Name", "Index","Par 71", "Strokes"])
		headers = ["\nName", "\nIndex", "CWV HCP\nWhites", "CWV\n Strokes"] #, "TPC HCP\n Par 72", "TPC-72\nStrokes" ]		
		column_print_alignment = list(("right","right","right","right"))
		course_info = f"CWV: CR = {cwv_rating_white_71} | SR = {cwv_slope_white_71} | Par = {cwv_hcp_71}"

	else: print('Bad Choice')
	
	#Finish printout:
	print('-------------------------------------------------------')
	print(" \n Today's date:", today)
	print()
	print (tabulate(results_list, tablefmt='simple', colalign=column_print_alignment, headers=headers, floatfmt=".1f"))
	print ()
	print (course_info)
	print ('Course Handicap = (H.I. x SR / 113) + (CR - Par)')
	print ()
	# print('--------------------------------------------------------')
	for player in player_list:
		if player.playing:
			print (player.email)
	print("\n\n")
	#END-----------------PRINT OUT TABLE OF INDEXES / HANDICAPS / STROKES OFF LOW HANDICAP----------		


In [15]:
def main():
	#Update the indexes of all players from the excel file
	get_indexes_from_xl_file()
	day_of_play = input("[M]onday, [W]ednesday, or [F]riday? >")
	update_player_status(day_of_play)

	#accumulate list of handicaps for sorting purposes
	tpc_list_70 = []
	tpc_list_72 = [] 
	cwv_list = [] 

	#TODO:  Complete the Smartsheet sign-up list to update player.playing

	for player in player_list:
		if player.playing:
			tpc_list_70.append(player.handicap_tpc_70)
			tpc_list_72.append(player.handicap_tpc_72)
			cwv_list.append(player.handicap_cwv_71)


	#determine lowest handicaps
	tpc_list_70.sort()
	tpc_min_70 = tpc_list_70[0] #lowest TPC70 handicap
	# print(f"TPC 70 Min: {tpc_min_70}")

	tpc_list_72.sort()
	tpc_min_72 = tpc_list_72[0] #lowest TPC72 handicap
	# print(f"TPC 72 Min: {tpc_min_72}")

	cwv_list.sort()
	cwv_min = cwv_list[0] #lowest CWV handicap
	# print(f"CWV Min: {cwv_min}")

	print_results_new(tpc_min_70,tpc_min_72, cwv_min)

Try to read / print out the spreadsheet using Pandas

In [16]:
def get_indexes_from_xl_using_pandas():
    df = pd.read_excel('Handicap Index Course Handicap Report.xlsx', sheet_name='Sheet1')
    print('Sheet Loaded')
    
    for ind in range(19):
        golfer_name = df['Golfer Name'][ind]
        h_i = (float(df['H.I.'][ind]))
        print(f"Golfer Name: {golfer_name}, \nHI: {h_i}")

    #Update handicap info in Player Class, set playing to no.
        for player in player_list:
#             print(f"Entering player list with GN: {golfer_name}, PN: {player.ghin_name}")
            if golfer_name == player.ghin_name:
                print(f"Processing: {golfer_name}")
                player.h_i = h_i
                player.class_tpc_white_70()
                player.class_tpc_white_72()
                player.class_cwv_white_71()
                player.playing = False


    

In [17]:
def main2():
    #Update the indexes of all players from the excel file
	get_indexes_from_xl_using_pandas()
	day_of_play = input("[M]onday, [W]ednesday, or [F]riday? >")
	update_player_status(day_of_play)

	#accumulate list of handicaps for sorting purposes
	tpc_list_70 = []
	tpc_list_72 = [] 
	cwv_list = [] 

	#TODO:  Complete the Smartsheet sign-up list to update player.playing

	for player in player_list:
		if player.playing:
			tpc_list_70.append(player.handicap_tpc_70)
			tpc_list_72.append(player.handicap_tpc_72)
			cwv_list.append(player.handicap_cwv_71)


	#determine lowest handicaps
	tpc_list_70.sort()
	tpc_min_70 = tpc_list_70[0] #lowest TPC70 handicap
	# print(f"TPC 70 Min: {tpc_min_70}")

	tpc_list_72.sort()
	tpc_min_72 = tpc_list_72[0] #lowest TPC72 handicap
	# print(f"TPC 72 Min: {tpc_min_72}")

	cwv_list.sort()
	cwv_min = cwv_list[0] #lowest CWV handicap
	# print(f"CWV Min: {cwv_min}")

	print_results_new(tpc_min_70,tpc_min_72, cwv_min)

In [18]:
main2()

Sheet Loaded
Golfer Name: Reid Baker, 
HI: 11.4
Processing: Reid Baker
Golfer Name: Bill Barnard, 
HI: 16.5
Processing: Bill Barnard
Golfer Name: Rick Baumgarth, 
HI: 11.4
Processing: Rick Baumgarth
Golfer Name: Rick Besse, 
HI: 16.1
Processing: Rick Besse
Golfer Name: Frank Broyles, 
HI: 16.2
Processing: Frank Broyles
Golfer Name: Jack Carroll, 
HI: 10.0
Processing: Jack Carroll
Golfer Name: Will Davis, 
HI: 15.6
Processing: Will Davis
Golfer Name: Rocky Duron, 
HI: 16.2
Processing: Rocky Duron
Golfer Name: Kent Fannon, 
HI: 15.3
Processing: Kent Fannon
Golfer Name: Curt Fitzgerald, 
HI: 18.2
Processing: Curt Fitzgerald
Golfer Name: Bob Heard, 
HI: 7.0
Processing: Bob Heard
Golfer Name: Richard Humphrey, 
HI: 17.6
Processing: Richard Humphrey
Golfer Name: Jim Sido, 
HI: 11.3
Processing: Jim Sido
Golfer Name: Dan Stewart, 
HI: 13.5
Processing: Dan Stewart
Golfer Name: Bill  Strand, 
HI: 11.2
Processing: Bill  Strand
Golfer Name: Larry Traub, 
HI: 8.9
Processing: Larry Traub
Golfer Name